In [2]:
import os
import pinecone
from getpass import getpass

from langchain.vectorstores import Pinecone
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import RetrievalQA

In [2]:
os.environ['OPENAI_API_KEY'] = getpass('OpenAI api key: ')
os.environ['PINECONE_API_KEY'] = getpass('Pinecone api key:')
os.environ['PINECONE_ENV'] = getpass('Pinecone environment (cloud region):')

In [3]:
embeddings = OpenAIEmbeddings()
pinecone.init(
    api_key=os.environ.get('PINECONE_API_KEY'),
    environment=os.environ.get('PINECONE_ENV')
)
docsearch = Pinecone.from_existing_index('lc0', embeddings)

In [4]:
chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(),
    retriever=docsearch.as_retriever()
)

In [5]:
query = 'Основные правила вкратце'
chain.run(query)

'Правила формирования системы управления рисками и внутреннего контроля для банков второго уровня определяют порядок формирования системы управления рисками и внутреннего контроля банков второго уровня. Система внутреннего контроля банка основывается на участии всех структурных подразделений и работников банка и организации внутреннего контроля как повседневной деятельности на всех уровнях управления. Основные принципы и обязанности совета директоров банка включают рациональное принятие решений и действие в интересах банка, добросовестное принятие решений и действие в интересах банка, активное вовлечение в деятельность банка и осведомленность о существенных изменениях деятельности банка и внешних условий.'